In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from __future__ import division, print_function, absolute_import
from tf_utils import load_features_with_deltas_stacking, random_mini_batches

train_data, test_data, train_labels, test_labels = load_features_with_deltas_stacking()
train_data = train_data.astype(np.float32)
test_data = test_data.astype(np.float32)

tf.logging.set_verbosity(tf.logging.INFO)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [4]:
#Seeding
seed = 3

# Network Parameters
input_h = train_data.shape[1] #Height
input_w = train_data.shape[2] #Width
num_classes = 2

In [5]:
def sound_net(features, labels, mode):

    # Input Layer
    x = tf.reshape(features["x"], [-1, input_h, input_w, 1])

    # Convolutional Layer #1
    # input shape [batch, input_h, input_w, 1]
    # output shape [batch, input_h, input_w, 80]   
    conv1 = tf.layers.conv2d(inputs=x, filters=80, kernel_size=[57, 6], padding="same", activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], padding="same", strides=2)
    pool1 = tf.layers.dropout(inputs=pool1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Convolutional Layer #2
    #input shape [batch_size, input_h/2, input_w/2, 80]
    #output shape [batch_size, input_h/2, input_w/2, 80]
    conv2 = tf.layers.conv2d(inputs=pool1, filters=80, kernel_size=[1, 3], padding="same", activation=tf.nn.relu)
    
    # Pooling Layer #2
    # Input Tensor Shape: [batch_size, input_h/2, input_w/2, 80]
    # Output Tensor Shape: [batch_size, input_h/16 input_w/14, 80]    
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[8, 7], padding="same", strides=[8, 7])

    # Fully connected layer #1
    # Reshape conv2 output to fit fully connected layer input
    pool2_flat = tf.reshape(pool2, [-1, int(input_h/16)*int(input_w/14)*80])    
    dense1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dense1 = tf.layers.dropout(inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Fully connected layer #2
    dense2 = tf.layers.dense(inputs=dense1, units=1024, activation=tf.nn.relu)
    dense2 = tf.layers.dropout(inputs=dense2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)    
    
    # Logits layer
    logits = tf.layers.dense(inputs=dense2, units=num_classes)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = { "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]) }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [15]:
def main(unused_argv):

    # Create the Estimator or warm-start the last one
    sound_classifier = tf.estimator.Estimator(
      model_fn=sound_net, model_dir="/tmp/sound_classifier_model")

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
#     tensors_to_log = {"probabilities": "softmax_tensor"}
#     logging_hook = tf.train.LoggingTensorHook(
#       tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=64,
      num_epochs=None,
      shuffle=True)
    
    sound_classifier.train(
      input_fn=train_input_fn,
      steps=200)
#     ,hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data},
      y=test_labels,
      num_epochs=1,
      shuffle=False)
    eval_results = sound_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [16]:
if __name__ == "__main__":
    tf.app.run()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/sound_classifier_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbd9c445550>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/sound_classifier_model/model.ckpt-202
INFO:tensorflow:Saving checkpoints for 203 into /tmp/sound_classifier_model/model.ckpt.
INFO:tensorflow:loss = 0.71045405, step = 203
INFO:tensorflow:global_step/sec: 5.04933
INFO:tensorflow:loss = 0.5413848, step = 303 (19.806 sec)
INFO:tensorflow:Saving checkpoints for 402 into 

UnboundLocalError: local variable 'train_op' referenced before assignment

In [8]:
tf.reset_default_graph()